#### N0 - 일반 정상 대화 데이터셋 생성기

이 스크립트는 **OpenAI API**를 사용해 **자연스러운 한국어 일상 대화 데이터셋**을 생성합니다.  
총 **502개의 시나리오**를 기반으로, 각 시나리오당 1~2개의 대화를 만들어 **약 1,000개 데이터**를 생성합니다.

---

✅ 주요 기능
- **자연스러운 한국인 이름 자동 생성**
  - `KoreanNameGenerator`를 이용해 중복 없는 자연스러운 이름을 자동 생성
- **100개 단위로 분할 저장**
  - 1,000개 데이터를 10개 JSON 파일로 나누어 저장
- **API 실 호출 기반 생성**
  - 실제 OpenAI API 호출로 데이터 생성.
  - 총 토큰 사용량 출력

---

⚙️ 실행 순서
1. API 키 입력 → OpenAI 클라이언트 초기화  
2. 시나리오 목록 로드 (502개)  
3. 이름 자동 생성 (중복 방지)  
4. 대화 생성 (시나리오별 1~2개)  
5. 100개 단위로 JSON 파일 저장  
6. 총 토큰 사용량 출력  

In [1]:
import os
import json
import time
import random
from typing import Dict, List, Optional, Set
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
from datetime import datetime

1. 설정 및 상수

In [2]:
# API 설정
OPENAI_MODEL = 'gpt-4o'
API_DELAY_SECONDS = 1.0

# 생성 설정
TOTAL_TARGET_SAMPLES = 1000  # 총 생성할 데이터 수
BATCH_SIZE = 100  # 배치당 데이터 수
NUM_BATCHES = 10  # 총 배치 수

# 출력 설정
OUTPUT_DIR = "n0_generated_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

2. 한국인 이름 생성기

In [ ]:
import random
from typing import List, Set

class KoreanNameGenerator:
    """중복 없는 자연스러운 한국인 이름 생성기 (성과 이름 첫 글자 중복 방지)"""

    # 한국 성씨
    SURNAMES = [
        "김", "이", "박", "최", "정", "강", "조", "윤", "장", "임",
        "한", "오", "서", "신", "권", "황", "안", "송", "전", "홍",
        "유", "고", "문", "양", "손", "배", "백", "허", "남", "심",
        "노", "하", "곽", "성", "차", "주", "우", "구", "민", "류"
    ]

    # 이름 200개
    GIVEN_NAMES = [
        "민석", "서준", "도윤", "하준", "주원", "시우", "지호", "지후", "유준", "현우",
        "하윤", "준우", "예준", "건우", "승우", "민재", "연우", "준호", "시윤", "현준",
        "예진", "하연", "유주", "소윤", "유진", "민서", "지유", "세은", "주하", "다현",
        "주호", "지훈", "준혁", "하민", "도훈", "지헌", "하성", "유찬", "민호", "승호",
        "서연", "하은", "지우", "서윤", "지민", "수아", "하린", "예린", "채원", "유나",
        "지원", "지윤", "다은", "수빈", "아린", "하율", "예은", "서하", "우빈", "성현",
        "도현", "시현", "태현", "민성", "승민", "지안", "현민", "나은", "가은", "하영",
        "시연", "서우", "예원", "민하", "채윤", "예서", "승현", "진우", "은우", "태윤",
        "현서", "우진", "재윤", "성민", "재원", "지환", "지현", "소율", "하늘", "은서",
        "서희", "예나", "수연", "서린", "유리", "라윤", "지한", "도연", "태민", "하진",
        "윤호", "하람", "지혁", "민결", "은찬", "유담", "준영", "재민", "하온", "윤서",
        "서진", "은호", "민우", "재현", "예담", "지완", "유성", "세윤", "나윤", "다온",
        "서빈", "도하", "현진", "윤재", "가율", "하빈", "은재", "민규", "유건", "시훈",
        "태율", "민솔", "도겸", "유하", "하경", "주은", "태린", "지율", "현이", "민채",
        "서현", "라희", "하엘", "지영", "은별", "도아", "예슬", "나리", "수현", "가빈",
        "세영", "연진", "나율", "지효", "라온", "민교", "서유", "예솔", "주리", "윤아",
        "은유", "유민", "도유", "하음", "시온", "유현", "예아", "연서", "하예", "재하",
        "도율", "하랑", "서담", "지유", "라미", "나현", "가현", "채린", "예온", "시안",
        "도진", "지혁", "은지", "민호", "유찬", "하율", "나윤", "가온", "도은", "예린"
    ]

    def __init__(self):
        self.used_names: Set[str] = set()
        self.generation_count = 0

    def generate_name(self, max_attempts: int = 100) -> str:
        """중복 없는 한국인 이름 생성 (성과 이름 첫 글자 중복 방지)"""
        for _ in range(max_attempts):
            surname = random.choice(self.SURNAMES)

            # 이름 후보 중에서 성의 첫 글자와 다르게 시작하는 이름만 필터링
            valid_given_names = [name for name in self.GIVEN_NAMES if not name.startswith(surname)]
            if not valid_given_names:
                valid_given_names = self.GIVEN_NAMES  # 혹시 전부 제외될 경우 대비

            given_name = random.choice(valid_given_names)
            full_name = surname + given_name

            if full_name not in self.used_names:
                self.used_names.add(full_name)
                self.generation_count += 1
                return full_name

        # 중복 시 숫자 추가 (fallback)
        surname = random.choice(self.SURNAMES)
        given_name = random.choice(self.GIVEN_NAMES)
        full_name = f"{surname}{given_name}{self.generation_count}"
        self.used_names.add(full_name)
        self.generation_count += 1
        return full_name

    def generate_names(self, count: int) -> List[str]:
        """여러 개의 중복 없는 이름 생성"""
        return [self.generate_name() for _ in range(count)]

    def reset(self):
        """사용된 이름 초기화"""
        self.used_names.clear()
        self.generation_count = 0


# 전역 이름 생성기 초기화
name_generator = KoreanNameGenerator()

print("✅ 한국인 이름 생성기 초기화 완료")
print("\n테스트 생성 이름 10개:")
for i, name in enumerate(name_generator.generate_names(10), 1):
    print(f"  {i}. {name}")

✅ 한국인 이름 생성기 초기화 완료

테스트 생성 이름 10개:
  1. 고승민
  2. 송승호
  3. 유하연
  4. 임채원
  5. 강승현
  6. 서승현
  7. 서예서
  8. 차하영
  9. 곽태윤
  10. 한시우


3. 토큰 추적기

In [4]:
class TokenTracker:
    """토큰 사용량 추적 클래스"""
    
    def __init__(self):
        self.total_prompt_tokens = 0
        self.total_completion_tokens = 0
        self.total_tokens = 0
        self.request_count = 0
        self.batch_stats = []  # 배치별 통계
    
    def add_usage(self, usage):
        """API 응답에서 토큰 사용량 추가"""
        self.total_prompt_tokens += usage.prompt_tokens
        self.total_completion_tokens += usage.completion_tokens
        self.total_tokens += usage.total_tokens
        self.request_count += 1
    
    def add_batch_stats(self, batch_num: int, samples: int, tokens: int):
        """배치 통계 추가"""
        self.batch_stats.append({
            'batch': batch_num,
            'samples': samples,
            'tokens': tokens
        })
    
    def get_summary(self):
        """사용량 요약 반환"""
        return {
            'total_requests': self.request_count,
            'prompt_tokens': self.total_prompt_tokens,
            'completion_tokens': self.total_completion_tokens,
            'total_tokens': self.total_tokens,
            'avg_tokens_per_request': round(self.total_tokens / self.request_count, 2) if self.request_count > 0 else 0
        }
    
    def print_summary(self):
        """사용량 요약 출력"""
        summary = self.get_summary()
        print("\n" + "="*80)
        print("📊 토큰 사용량 요약")
        print("="*80)
        print(f"총 API 요청 수: {summary['total_requests']:,}회")
        print(f"\n토큰 사용량:")
        print(f"  • 입력 토큰:    {summary['prompt_tokens']:,}")
        print(f"  • 출력 토큰:    {summary['completion_tokens']:,}")
        print(f"  • 총 토큰:      {summary['total_tokens']:,}")
        print(f"  • 평균/요청:    {summary['avg_tokens_per_request']:,.2f}")
        
        if self.batch_stats:
            print(f"\n배치별 통계:")
            for stat in self.batch_stats:
                print(f"  배치 {stat['batch']:2d}: {stat['samples']:3d}개 샘플, {stat['tokens']:,} 토큰")
        
        print("="*80)


# 전역 토큰 추적기
token_tracker = TokenTracker()
print("✅ 토큰 추적기 초기화 완료")

✅ 토큰 추적기 초기화 완료


4. 시나리오 로드

In [5]:
# 시나리오 로드
with open('n0_scenarios.json', 'r', encoding='utf-8') as f:
    ALL_SCENARIOS = json.load(f)

print(f"✅ 총 {len(ALL_SCENARIOS)}개의 시나리오를 로드했습니다.")
print(f"\n샘플 시나리오 (처음 3개):")
for i, s in enumerate(ALL_SCENARIOS[:3]):
    print(f"  {i+1}. {s['type']} - {s['context']}")

✅ 총 502개의 시나리오를 로드했습니다.

샘플 시나리오 (처음 3개):
  1. 친구 간 약속 잡기 - 주말 영화 보기 계획
  2. 가족 안부 전화 - 부모님 건강 확인
  3. 동료와 점심 메뉴 결정 - 오늘 점심 뭐 먹을지 상의


5. 시스템 프롬프트

In [6]:
SYSTEM_INSTRUCTION = """당신은 일상적인 전화 대화 데이터 생성 전문가입니다.

**목적**: 일반인들의 평범한 일상 전화 대화를 생성합니다.

**출력 형식**: 반드시 유효한 JSON만 출력하세요.
```json
{
  "dialogue": "발화1\\n발화2\\n발화3...",
  "label": 0,
  "meta": {
    "conversation_type": "casual_normal",
    "relationship": "친구",
    "context": "약속 잡기",
    "tone": "친근함",
    "topic": "주말 만남",
    "emotion": "즐거움",
    "call_environment": "(아래 옵션 중 하나를 랜덤하게 선택)"
  }
}
```

**call_environment 옵션 (균등한 비율로 사용되도록 다양한 선택을 할 것)**:
1. "시끄러운 실내" - 카페, 식당, 쇼핑몰, 헬스장
2. "조용한 실내" - 집, 사무실, 개인 공간
3. "실외" - 거리, 공원, 주차장
4. "이동 중" - 걷는 중, 버스/지하철 안, 차 안
5. "조용한 공공장소" - 도서관, 병원 대기실, 은행
- call_environment는 위 다섯 가지 중 하나를 무작위(random)로 선택할 것.
- **균등 확률(각 20%)**로 선택되도록 가정하며, 특정 항목이 반복적으로 선택되지 않도록 주의한다.
- 여러 샘플을 생성할 때는 내부적으로 주사위를 굴리듯 난수를 기반으로 분포를 제어한다고 생각하고,
한 세션 내에서는 동일한 환경이 연속적으로 나타나지 않도록 한다.
- 무작위성은 문맥보다 우선하며, 각 환경의 표현이 자연스럽게 드러나도록 배경 묘사를 조정한다.

**환경별 대화 특징 (자연스럽게 반영)**:
- 조용한 실내: 편안하게 대화, 배경음 없음
- 시끄러운 실내: "거기 시끄럽네", "음 잘 안 들려", "잠깐만 조용한 데로"
- 실외: "바람 불어", "날씨 좋다", "밖에 나와 있어"
- 이동 중: **하나의 이동 수단만 선택하여 일관되게 사용**
  → 도보: "지금 걷는 중이야", "잠깐 신호등에서", "횡단보도 건너면서"
  → 대중교통: "버스 타고 가는데", "지하철에서 전화하는 중", "다음 정거장에서"
  → 차량: "운전하면서 받았어", "신호 대기 중이야", "주차하고 전화할게"
- 조용한 공공장소: 목소리 낮춤, "조용히 해야 돼", 짧고 간결하게


**핵심 규칙**:
1. **label = 0 (정상 대화)**
2. 민감정보 절대 없음 (계좌번호, 주민번호 등 언급 금지)
3. STT 형식: 소문자, 특수문자 없음, 발화는 \\n으로 구분
4. **절대 금지**: 전화 대화에서 이름 자기소개 금지 ("안녕하세요 저는 OOO입니다" 같은 표현 사용 금지)
5. **call_environment는 5가지 옵션을 골고루 사용** - 특정 환경에 편중되지 않도록 균등 분배

6. **진짜 일상 대화처럼**:
   - 간투사 자연스럽게: "음", "어", "그", "아", "저" (자주)
   - 반말/존댓말 관계에 맞게
   - 웃음소리: "하하", "히히", "크크" (가끔)
   - 망설임: "그게 말이야", "음 그러니까", "어 뭐더라"
   - 상대 말 받아주기: "그러게", "맞아", "진짜", "대박"

7. **대화 시작 방식 (관계별)**:
   
   **친구/가족 (비격식)**:
   - "야 뭐해", "어 나야", "음 별일 없지"
   - "있잖아", "저기 말인데"
   
   **고객↔업체 (격식, 하지만 자기소개 NO)**:
   - 고객: "거기 OO 맞죠", "주문하려고 하는데요", "예약 변경하려고요"
   - 업체: "네 말씀하세요", "안녕하세요 무엇을 도와드릴까요", "여기 OO입니다"
   - ❌ 금지: "안녕하세요 저는 김OO입니다", "최OO고객님 안녕하세요"
   
   **학생↔교수/병원 (존중)**:
   - 학생: "교수님 안녕하세요", "지금 괜찮으시면", "질문이 있어서요"
   - 교수: "네 말해보세요", "어 OO 학생", "무슨 일이에요"
   - 필요한 경우 자기 소개를 포함

8. **대화 흐름 패턴**:
   **도입 (1-2턴)**:
   - 관계에 맞는 자연스러운 시작
   - 용건 제시

   **본론 (4-7턴)**:
   - 주제 전개: 구체적 내용 논의
   - 의견 교환: "나는 말이야", "음 그건 좀"
   - 주제에 대해 충분히 이야기하고, 세부 사항도 다룰 것
   - 관련된 경험이나 생각을 자연스럽게 공유
   - 농담/잡담 섞임

   **마무리 (2-3턴)**:
   - 정리: "그럼", "알았어", "오케이"
   - 작별: "나중에 봐", "연락할게", "감사합니다"

9. **통화 환경 디테일**:
   - 배경 언급: "지금 집이야", "카페에서 전화하는 중"
   - 소음 반응: "거기 시끄럽네", "음 잘 안 들려"
   - 멀티태스킹: "밥 먹으면서 받았어", "걸어가면서 말하는 거야"

10. **감정 표현 (자연스럽게)**:
   - 즐거움: "오 좋다", "완전 좋지", "하하 그러게"
   - 공감: "아 진짜", "맞아 맞아", "나도 그랬어"
   - 걱정: "음 괜찮아", "너무 걱정 마"
   - 짜증: "아 진짜", "그러니까 말이야"

11. **관계별 특징**:
   - 친구: 반말, 장난, 편함
   - 가족: 다정함, 걱정, 잔소리
   - 동료: 존댓말 또는 편한 반말
   - 업체: 공손한 존댓말

12. **대화 주제 (예시)**:
   - 약속: "언제 만날까", "어디서 볼까"
   - 안부: "요즘 어때", "잘 지내"
   - 날씨: "오늘 덥더라", "비 온대"
   - 음식: "뭐 먹을까", "거기 맛있어"
   - 취미: "영화 봤어", "운동 가자"
   - 가벼운 고민: "어떻게 생각해", "너는 어때"

13. **개인정보 보호 원칙 (엄격히 준수)**:

   **A. 개인정보 요청 최소화**:
   - 기본 원칙: 개인정보가 **불필요하면 절대 묻지 않음**
   - 예시:
     * 배달 주문: 주소, 연락처만 (주민번호, 생년월일 불필요)
     * 헬스장 문의: 이름, 연락처만 (상세 개인정보 불필요)
     * 택배 확인: 송장번호만 (개인정보 불필요)

   **B. 필요 시에도 최소한만 요청**:
   - 병원 예약: "생년월일 뒤 4자리만", "성함과 생년월일만"
   - 은행 문의: "카드번호 뒤 4자리", "예금주명만"
   - 본인 확인: 부분 정보로 충분

   B-1. 생년월일 뒤 4자리 형식: MMDD(월월일일, MM=01~12, DD=01~31만 가능)
   B-2. 전화번호 뒤 4자리 형식: 반드시 4자리 숫자
   B-3. 모든 숫자 정보는 자릿수와 일반 규칙을 정확히 준수할 것
   B-4. 상황별 구분
   a. 배달/예약 서비스 (전체 정보 필수)**:
   - 배달 음식: "주소 전체와 연락처 전체" 필요
   - 병원/미용실 예약: "이름, 생년월일, 연락처 전체" 필요
   - 택배 발송: "주소 전체, 받는 사람 이름, 연락처 전체" 필요
   - **이유**: 서비스 제공을 위해 완전한 정보가 필수
   - **주소 형식**: "서울시 강남구 대치동 123번지 101호" 같이 구체적으로
   - **연락처 형식**: "010-1234-5678" 전체 번호
   b. 조회/확인 서비스 (부분 정보로 본인 확인)**:
   - 택배 조회: "송장번호" 또는 "전화번호 뒷자리 4자리"
   - 예약 확인/변경: "이름과 생년월일 뒷자리" 또는 "예약번호"
   - 은행 문의: "계좌번호 뒷자리" 또는 "카드번호 뒷자리"
   - **이유**: 이미 등록된 정보 조회이므로 부분 정보로 본인 확인
   c. 숫자 정보 형식 규칙 (정확하게 준수)**:
   - 생년월일 뒷자리: MMDD (예: 0315, 1225)
   - 전화번호 뒷자리: 4자리 숫자 (예: 1234)
   - 주민번호: **절대 언급 금지**
   - 계좌번호/카드번호: 전체 또는 뒷자리만
   d. 절대 금지:
   - "지갑에 적힌", "메모해둔" 같은 이상한 표현
   - 애매한 주소: "아파트 근처", "역 앞" → 구체적인 주소 사용
   - 주민번호 13자리 전체
   - 계좌번호 + 비밀번호 조합

   **C. 요청 시 반드시 이유 설명**:
   - "본인 확인을 위해 생년월일 뒤 4자리만 부탁드립니다"
   - "배송 확인용으로 연락처 뒷자리만 말씀해주시겠어요"
   - "예약 조회를 위해 성함만 확인할게요"

   **D. 고객이 과도하게 제공하려 할 때**:
   - "아니요 뒷자리만으로 충분합니다"
   - "전체는 필요 없고 일부만 말씀해주세요"
   - "개인정보 보호를 위해 최소한만 받고 있습니다"

   **E. 절대 요청하지 않는 정보**:
   - 주민번호 전체 13자리
   - 계좌번호 + 비밀번호 조합
   - 카드번호 전체 + CVC + 비밀번호
   - 불필요한 가족 정보
   - 건강정보 (병원 외 상황)

14. 대화 길이: 6-20턴 (짧고 간결하되 자연스럽고 충분한 대화)
15. 자연스럽고 편안한 분위기
16. 플레이스홀더 금지
17. 화자 표시 없이 발화만 출력
18. 업무/보안과 무관한 순수 일상 대화
"""

print("✅ 시스템 프롬프트 로드 완료")

✅ 시스템 프롬프트 로드 완료


6. OpenAI 클라이언트 초기화

In [7]:
def initialize_client() -> Optional[OpenAI]:
    """OpenAI 클라이언트 초기화 (input으로 키 입력)"""
    
    # 환경변수에 없으면 직접 입력 요청
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        api_key = input("🔑 OpenAI API 키를 입력하세요: ").strip()

    if not api_key:
        print("❌ API 키가 입력되지 않았습니다.")
        return None

    try:
        client = OpenAI(api_key=api_key)
        print("✅ OpenAI 클라이언트 초기화 완료")
        return client
    except Exception as e:
        print(f"❌ 클라이언트 초기화 실패: {e}")
        return None


# 실행 시 입력 요청됨
client = initialize_client()

✅ OpenAI 클라이언트 초기화 완료


7. 대화 생성 함수

In [8]:
def generate_single_conversation(
    client: OpenAI,
    scenario: Dict,
    participant_names: List[str]
) -> Optional[Dict]:
    """단일 대화 생성"""
    
    # call_environment를 랜덤 선택
    call_environments = [
        "시끄러운 실내",
        "조용한 실내", 
        "실외",
        "이동 중",
        "조용한 공공장소"
    ]
    selected_environment = random.choice(call_environments)
    
    user_prompt = f"""
시나리오: {scenario['type']}
관계: {scenario['relationship']}
상황: {scenario['context']}
톤: {scenario['tone']}

대화 참여자 이름: {', '.join(participant_names)}

**중요: call_environment는 반드시 "{selected_environment}"로 설정할 것**

위 시나리오에 맞는 자연스러운 일상 전화 대화를 생성하세요.
대화에 이름을 직접 언급하지 말고, meta 정보의 participants에만 포함하세요.
"""
    
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_INSTRUCTION},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.9,
            response_format={"type": "json_object"}
        )
        
        # 토큰 사용량 추적
        token_tracker.add_usage(response.usage)
        
        # 응답 파싱
        content = response.choices[0].message.content
        data = json.loads(content)
        
        # 메타 정보에 참여자 추가
        if 'meta' not in data:
            data['meta'] = {}
        data['meta']['participants'] = participant_names
        
        # 시나리오 정보 추가
        data['scenario_type'] = scenario['type']
        data['scenario_relationship'] = scenario['relationship']
        data['scenario_context'] = scenario['context']
        
        return data
        
    except json.JSONDecodeError as e:
        print(f"  ⚠️  JSON 파싱 실패: {e}")
        return None
    except Exception as e:
        print(f"  ⚠️  생성 실패: {e}")
        return None


print("✅ 대화 생성 함수 정의 완료")

✅ 대화 생성 함수 정의 완료


8. 배치 생성 함수

In [9]:
def generate_batch(
    client: OpenAI,
    batch_num: int,
    scenarios: List[Dict],
    target_samples: int,
    delay: float = 1.0
) -> pd.DataFrame:
    """
    배치 단위로 대화 데이터 생성
    
    Args:
        client: OpenAI 클라이언트
        batch_num: 배치 번호
        scenarios: 사용할 시나리오 리스트
        target_samples: 목표 샘플 수
        delay: API 호출 간 딜레이
    """
    print(f"\n{'='*80}")
    print(f"📦 배치 {batch_num} 생성 시작 (시나리오 {len(scenarios)}개 → 목표 {target_samples}개)")
    print(f"{'='*80}\n")
    
    conversations = []
    batch_start_tokens = token_tracker.total_tokens
    start_time = time.time()  # 시작 시간 기록
    
    # 각 시나리오당 생성할 샘플 수 계산
    samples_per_scenario = target_samples // len(scenarios)
    remaining_samples = target_samples % len(scenarios)
    
    for idx, scenario in enumerate(scenarios, 1):
        # 이 시나리오에서 생성할 샘플 수
        num_samples = samples_per_scenario
        if idx <= remaining_samples:
            num_samples += 1
        
        if num_samples == 0:
            continue
        
        scenario_success = 0
        scenario_fail = 0
        scenario_names = []  # 생성된 이름들 저장
        
        for sample_idx in range(num_samples):
            # 참여자 이름 생성 (2명)
            participant_names = name_generator.generate_names(2)
            
            # 대화 생성
            conversation = generate_single_conversation(
                client=client,
                scenario=scenario,
                participant_names=participant_names
            )
            
            if conversation:
                conversations.append(conversation)
                scenario_success += 1
                scenario_names.append(f"({', '.join(participant_names)})")
            else:
                scenario_fail += 1
            
            # API 호출 간 딜레이
            time.sleep(delay)
        
        # 시나리오별 결과를 한 줄로 출력 (이름 포함)
        status = "✅" if scenario_fail == 0 else "⚠️"
        names_str = " ".join(scenario_names) if scenario_names else ""
        print(f"{status} [{idx}/{len(scenarios)}] {scenario['type']}: {scenario_success}개 생성 {names_str}" + 
              (f" ({scenario_fail}개 실패)" if scenario_fail > 0 else ""))
        
        # 진행바 + 예상 시간 표시
        if idx % 5 == 0 or idx == len(scenarios):
            progress = idx / len(scenarios)
            bar_length = 40
            filled = int(bar_length * progress)
            bar = '█' * filled + '░' * (bar_length - filled)
            
            # 경과 시간 및 예상 남은 시간 계산
            elapsed_time = time.time() - start_time
            avg_time_per_scenario = elapsed_time / idx
            remaining_scenarios = len(scenarios) - idx
            estimated_remaining = avg_time_per_scenario * remaining_scenarios
            
            # 시간 포맷팅
            elapsed_str = f"{int(elapsed_time//60)}:{int(elapsed_time%60):02d}"
            if estimated_remaining > 0:
                remaining_str = f"{int(estimated_remaining//60)}:{int(estimated_remaining%60):02d}"
                time_info = f"[{elapsed_str}<{remaining_str}, {avg_time_per_scenario:.2f}s/it]"
            else:
                time_info = f"[{elapsed_str}, {avg_time_per_scenario:.2f}s/it]"
            
            print(f"📊 배치 {batch_num}: [{bar}] {progress*100:.1f}% ({idx}/{len(scenarios)}) {time_info} | 생성: {len(conversations)}개\n")
    
    # 배치 통계
    batch_tokens = token_tracker.total_tokens - batch_start_tokens
    token_tracker.add_batch_stats(batch_num, len(conversations), batch_tokens)
    
    total_time = time.time() - start_time
    print(f"⭐ 배치 {batch_num} 완료: {len(conversations)}/{target_samples}개 생성, 토큰: {batch_tokens:,}, 소요시간: {int(total_time//60)}분 {int(total_time%60)}초\n")
    
    # DataFrame 생성
    if conversations:
        df = pd.DataFrame(conversations)
        return df
    else:
        return pd.DataFrame()


print("✅ 배치 생성 함수 정의 완료")

✅ 배치 생성 함수 정의 완료


9. 데이터 저장 함수

In [10]:
def save_batch_data(df: pd.DataFrame, batch_num: int, output_dir: str):
    """배치 데이터 저장"""
    if df.empty:
        print(f"⚠️  배치 {batch_num}: 저장할 데이터 없음")
        return
    
    # 기본 버전 (dialogue, label만)
    basic_df = df[['dialogue', 'label']].copy()
    basic_file = os.path.join(output_dir, f"n0_batch_{batch_num:02d}_basic.csv")
    basic_df.to_csv(basic_file, index=False, encoding='utf-8-sig')
    
    # 상세 버전 (모든 정보)
    detailed_file = os.path.join(output_dir, f"n0_batch_{batch_num:02d}_detailed.csv")
    df.to_csv(detailed_file, index=False, encoding='utf-8-sig')
    
    # JSON 버전
    json_file = os.path.join(output_dir, f"n0_batch_{batch_num:02d}.json")
    df.to_json(json_file, orient='records', force_ascii=False, indent=2)
    
    print(f"💾 배치 {batch_num} 저장 완료:")
    print(f"   - {basic_file}")
    print(f"   - {detailed_file}")
    print(f"   - {json_file}")


print("✅ 데이터 저장 함수 정의 완료")

✅ 데이터 저장 함수 정의 완료


10. 메인 실행 - 데이터 생성

In [11]:
def main_generation():
    """메인 생성 프로세스"""
    
    if not client:
        print("❌ OpenAI 클라이언트가 초기화되지 않았습니다.")
        return
    
    print("\n" + "="*80)
    print("🚀 N0 데이터 생성 시작")
    print("="*80)
    print(f"전체 시나리오: {len(ALL_SCENARIOS)}개")
    print(f"목표 데이터: {TOTAL_TARGET_SAMPLES}개")
    print(f"배치 수: {NUM_BATCHES}개 (각 {BATCH_SIZE}개)")
    print(f"출력 디렉토리: {OUTPUT_DIR}")
    print("="*80)
    
    start_time = time.time()
    all_dataframes = []
    
    # 시나리오를 배치별로 분할
    scenarios_per_batch = len(ALL_SCENARIOS) // NUM_BATCHES
    
    for batch_num in range(1, NUM_BATCHES + 1):
        # 이 배치에서 사용할 시나리오 범위
        start_idx = (batch_num - 1) * scenarios_per_batch
        
        if batch_num == NUM_BATCHES:
            # 마지막 배치는 나머지 모두 사용
            end_idx = len(ALL_SCENARIOS)
        else:
            end_idx = start_idx + scenarios_per_batch
        
        batch_scenarios = ALL_SCENARIOS[start_idx:end_idx]
        
        # 배치 생성
        batch_df = generate_batch(
            client=client,
            batch_num=batch_num,
            scenarios=batch_scenarios,
            target_samples=BATCH_SIZE,
            delay=API_DELAY_SECONDS
        )
        
        # 배치 데이터 저장
        if not batch_df.empty:
            save_batch_data(batch_df, batch_num, OUTPUT_DIR)
            all_dataframes.append(batch_df)
        
        print(f"\n{'='*80}")
        print(f"✅ 배치 {batch_num}/{NUM_BATCHES} 완료")
        print(f"{'='*80}\n")
    
    # 전체 데이터 통합
    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        
        # 통합 데이터 저장
        combined_basic = os.path.join(OUTPUT_DIR, "n0_all_data_basic.csv")
        combined_detailed = os.path.join(OUTPUT_DIR, "n0_all_data_detailed.csv")
        combined_json = os.path.join(OUTPUT_DIR, "n0_all_data.json")
        
        combined_df[['dialogue', 'label']].to_csv(combined_basic, index=False, encoding='utf-8-sig')
        combined_df.to_csv(combined_detailed, index=False, encoding='utf-8-sig')
        combined_df.to_json(combined_json, orient='records', force_ascii=False, indent=2)
        
        print(f"\n💾 통합 데이터 저장 완료:")
        print(f"   - {combined_basic}")
        print(f"   - {combined_detailed}")
        print(f"   - {combined_json}")
        
        # 최종 통계
        print("\n" + "="*80)
        print("📊 최종 통계")
        print("="*80)
        print(f"총 생성 데이터: {len(combined_df)}개")
        
        # 대화 턴 수 통계
        combined_df['turn_count'] = combined_df['dialogue'].apply(lambda x: len(x.split('\n')))
        print(f"\n대화 턴 수:")
        print(f"  평균: {combined_df['turn_count'].mean():.1f}")
        print(f"  범위: {combined_df['turn_count'].min()} ~ {combined_df['turn_count'].max()}")
        
        # 이름 사용 통계
        print(f"\n생성된 고유 이름 수: {name_generator.generation_count}개")
        print(f"사용된 고유 이름 수: {len(name_generator.used_names)}개")
    
    # 토큰 사용량 요약
    token_tracker.print_summary()
    
    # 총 소요 시간
    elapsed_time = time.time() - start_time
    print(f"\n⏱️  총 소요 시간: {elapsed_time/60:.1f}분 ({elapsed_time:.1f}초)")
    print(f"\n✅ 전체 생성 완료!")
    print("="*80)


print("✅ 메인 함수 정의 완료")
print("\n실행하려면 다음 셀을 실행하세요.")

✅ 메인 함수 정의 완료

실행하려면 다음 셀을 실행하세요.


11. 실행

In [ ]:
# 실행
if __name__ == "__main__":
    main_generation()


🚀 N0 데이터 생성 시작
전체 시나리오: 502개
목표 데이터: 1000개
배치 수: 10개 (각 100개)
출력 디렉토리: n0_generated_data

📦 배치 1 생성 시작 (시나리오 50개 → 목표 100개)

✅ [1/50] 친구 간 약속 잡기: 2개 생성 (고서하, 류지훈) (강하율, 허현서)
✅ [2/50] 가족 안부 전화: 2개 생성 (강서윤, 차우진) (주연우, 민성현)
✅ [3/50] 동료와 점심 메뉴 결정: 2개 생성 (배현서, 임지유) (구주호, 남하연)
✅ [4/50] 배달 음식 주문: 2개 생성 (윤승우, 안하영) (백소율, 권예진)
✅ [5/50] 병원 예약 변경: 2개 생성 (임예나, 권소윤) (류민성, 송나은)
📊 배치 1: [████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 10.0% (5/50) [0:53<8:02, 10.72s/it] | 생성: 10개

✅ [6/50] 헬스장 문의: 2개 생성 (홍지현, 하성현) (손수연, 박지헌)
✅ [7/50] 지인에게 부탁: 2개 생성 (김시윤, 고다현) (임가은, 노승현)
✅ [8/50] 택배 배송 확인: 2개 생성 (서예준, 양유준) (우유리, 류우진)
✅ [9/50] 친구와 여행 계획: 2개 생성 (홍주원, 박지유) (권하준, 배건우)
✅ [10/50] 가족과 주말 일정 확인: 2개 생성 (안건우, 윤시현) (박지안, 류채윤)
📊 배치 1: [████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 20.0% (10/50) [2:07<8:28, 12.71s/it] | 생성: 20개

✅ [11/50] 동료 업무 외 잡담: 2개 생성 (심지후, 이하율) (민도훈, 오준혁)
✅ [12/50] 커피숍 예약 문의: 2개 생성 (황성현, 심예은) (배유나, 허은우)
✅ [13/50] 은행 업무 문의: 2개 생성 (백민성, 손수아) (성지안, 고승호)
✅ [14/50] 대학교 교수 상담: 2개 생성 (권유리, 하서윤) (문가은, 권진우)
✅ 